In [2]:
from datasets import load_dataset, concatenate_datasets, Dataset
import numpy as np
import torch
import torchcodec
from datasets import ClassLabel, Audio as DAudio
from IPython.display import display, Audio
new_labels = ['other', 'drone']



In [3]:
drone_audio_detection_from_hugging_face_dataset = load_dataset("geronimobasso/drone-audio-detection-samples")
drone_audio_detection_from_hugging_face_dataset = drone_audio_detection_from_hugging_face_dataset.cast_column("label", ClassLabel(names=new_labels))


In [4]:
hibou_dataset = load_dataset("audiofolder", data_dir="../data/raw/g")
hibou_dataset = hibou_dataset.cast_column("label", ClassLabel(names=new_labels))
# Remap values
hibou_dataset = hibou_dataset.map(lambda x: {"label": 1 - x["label"]})

In [5]:
label_feature = drone_audio_detection_from_hugging_face_dataset["train"].features["label"]
ad1 = drone_audio_detection_from_hugging_face_dataset["train"][17000]
print(label_feature.int2str(ad1["label"]))
display(Audio(ad1["audio"]["array"], rate=ad1["audio"]["sampling_rate"]))


label_feature = hibou_dataset["train"].features["label"]
ad1 = hibou_dataset["train"][1]
print(label_feature.int2str(ad1["label"]))
display(Audio(ad1["audio"]["array"], rate=ad1["audio"]["sampling_rate"]))

drone


drone


In [6]:
dataset = concatenate_datasets([drone_audio_detection_from_hugging_face_dataset["train"], hibou_dataset["train"]])

In [7]:
dataset2 = load_dataset("audiofolder", data_dir="../data/raw/dataset2")
dataset2 = dataset2.cast_column("label", ClassLabel(names=new_labels))
dataset2 = dataset2.map(lambda x: {"label": 1 - x["label"]})


In [8]:
label_feature = dataset2["train"].features["label"]
ad1 = dataset2["train"][200]
print(label_feature.int2str(ad1["label"]))
display(Audio(ad1["audio"]["array"], rate=ad1["audio"]["sampling_rate"]))


drone


In [9]:
dataset = concatenate_datasets([dataset, dataset2["train"]])

In [10]:
import pandas as pd
df = dataset.to_pandas()
label_counts = df["label"].value_counts().sort_index()
print(label_counts)
print(new_labels[0], new_labels[1])

label
0     22253
1    169517
Name: count, dtype: int64
other drone


In [11]:
import torchaudio
import torch

def split_audio_into_chunks(audio_array, sampling_rate, chunk_duration=0.5):
    # Calculate the number of samples per chunk
    samples_per_chunk = int(chunk_duration * sampling_rate)
    # Pad the audio if necessary to make it divisible by samples_per_chunk
    remainder = audio_array.shape[-1] % samples_per_chunk
    if remainder != 0:
        padding = samples_per_chunk - remainder
        audio_array = torch.nn.functional.pad(audio_array, (0, padding))
    # Split the audio into chunks
    chunks = audio_array.unfold(-1, samples_per_chunk, samples_per_chunk)
    return chunks

In [12]:
sample = dataset[2001]
audio_array = torch.tensor(sample["audio"]["array"]).float()
sampling_rate = sample["audio"]["sampling_rate"]
label = sample["label"]
display(Audio(audio_array, rate=sampling_rate))
chunks = split_audio_into_chunks(audio_array, sampling_rate)
for chunk in chunks:
    display(Audio(chunk, rate=sampling_rate))

In [14]:

chunk_dataset = dataset.map(
    split_and_flatten,
    batched=True,
    batch_size=16,         # tune this to fit RAM
    remove_columns=dataset.column_names,
)

In [15]:
#dataset.save_to_disk("../data/datasets/dataset1")

In [24]:
sample = chunk_dataset[2001]
audio_array = torch.tensor(sample["audio"]["array"]).float()
sampling_rate = sample["audio"]["sampling_rate"]
label = sample["label"]
display(Audio(audio_array, rate=sampling_rate))

sampling_rate

16000